In [40]:
import os
import time
import pandas as pd
from dotenv import load_dotenv
from supabase import create_client, Client

from langchain import PromptTemplate
from langchain_ollama import ChatOllama
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

# Replace with your Supabase project URL and public anon key
SUPABASE_URL = os.environ.get("SUPABASE_URL")
SUPABASE_KEY = os.environ.get("SUPABASE_KEY")

supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

In [27]:
def get_questions(top=5, live=False):
    
    if live:
        # Query the top 5 rows from the "aqua_rat_train" table
        response = supabase.table("aqua_rat_train").select("*").limit(top).execute()

        questions = response.data
        
        # save as pickle
        questions = pd.DataFrame.from_records(questions)

        # save pickle file
        questions.to_pickle("questions.pkl")
        
    else:
        # Load the questions from the pickle file
        questions = pd.read_pickle("questions.pkl")
        
    return questions


In [32]:
def get_concepts(live=False):
    """
    Fetches the list of math concepts either from the Supabase database (live mode)
    or from a locally saved pickle file (offline mode).

    Args:
        live (bool): If True, fetch concepts from the Supabase database. 
                     If False, load concepts from a local pickle file.

    Returns:
        DataFrame: A DataFrame containing the concepts.
    """
    if live:
        # Query the "concepts" table from Supabase
        response = supabase.table("concepts").select("*").filter("parent_id", "not.is.null", None).execute()

        # Extract concept names and create a DataFrame
        concepts = pd.DataFrame.from_records(response.data)

        # Save the DataFrame to a pickle file for offline use
        concepts.to_pickle("concepts.pkl")
        return concepts
    else:
        # Load the DataFrame from the pickle file
        concepts = pd.read_pickle("concepts.pkl")
        return concepts
    
# get_concepts(live=True)

In [83]:
questions_df = get_questions(live=True, top=100)
questions_df

,id,question,options,rationale,correct
0,1,"Two friends plan to walk along a 43-km trail, ...","[A)21, B)21.5, C)22, D)22.5, E)23]","If Q complete x kilometers, then P completes 1...",E
1,2,"In the coordinate plane, points (x, 1) and (5,...","[A)4 and 1, B)1 and 5, C)5 and 1, D)3 and 5, E...",Line k passes through the origin and has slope...,C
2,3,"For all numbers p and q, the operation @ is de...","[A)II, B)I and II, C)I and III, D)II and III, ...",p@q = p^2 - pq=p(p-q).... so p@q will be zero ...,B
3,4,Carl is facing very difficult financial times ...,"[A)$1600, B)$2000, C)$2150, D)$2500, E)$12000]","Usually, you are given the annual rate of inte...",A
4,5,The speed at which a man can row a boat in sti...,"[A)18 seconds, B)27 seconds, C)26 seconds, D)1...",Speed of the boat downstream = 25 +11\n= 36 km...,E
...,...,...,...,...,...
95,93,"A man, a woman and a boy can complete a job in...","[A)10, B)41, C)31, D)21, E)22]","Explanation:\n1 man’s 1 day’s work = 1/3, ¼ da...",B
96,94,"At present, the ratio between the ages of Amit...","[A)16, B)77, C)566, D)197, E)161]",Explanation:\nLet the present ages of Amit and...,A
97,95,A and B invests Rs.3000 and Rs.4000 respective...,"[A)9:6, B)9:8, C)9:1, D)9:9, E)9:5]",(3*6 + 6*6): (4*12)\n54:48 => 9:8.Answer:B,B
98,96,"If x-y=10, which of the following must be true...","[A)I only, B)II only, C)III only, D)I and II, ...",The best way to approach such questions is to ...,B


In [33]:
concepts_df = get_concepts(live=False)
concepts_df

,concept_id,name,parent_id,description
0,6,Real Number System,1,Properties and operations with real numbers
1,10,Exponents and Radicals,1,"Exponents, roots, and their properties"
2,14,Complex Numbers,1,Operations and equations with complex numbers
3,17,Vectors and Matrices,1,Operations with vectors and matrices
4,20,Sequences and Series,1,Patterns in sequences and series
5,23,Expressions and Equations,2,Manipulating and solving algebraic expressions
6,28,Absolute Value,2,Equations and inequalities involving absolute ...
7,32,Quadratic Equations and Functions,2,Quadratic expressions and their graphs
8,36,Polynomial and Rational Expressions,2,Operations with polynomials and rational funct...
9,39,Exponential and Logarithmic Functions,2,Exponential and logarithmic relationships


In [37]:
questions_to_process = questions_df.question.tolist()[:5]
questions_to_process

["Two friends plan to walk along a 43-km trail, starting at opposite ends of the trail at the same time. If Friend P's rate is 15% faster than Friend Q's, how many kilometers will Friend P have walked when they pass each other?",
 'In the coordinate plane, points (x, 1) and (5, y) are on line k. If line k passes through the origin and has slope 1/5, then what are the values of x and y respectively?',
 'For all numbers p and q, the operation @ is defined by p@q = p^2 - pq. If xy ≠ 0, then which of the following can be equal to zero?\nI. x@y\nII. (xy)@y\nIII. x@(x + y)',
 'Carl is facing very difficult financial times and can only pay the interest on a $10,000 loan he has taken. The bank charges him a quarterly compound rate of 4%. What is the approximate interest he pays annually?',
 'The speed at which a man can row a boat in still water is 25 kmph. If he rows downstream, where the speed of current is 11 kmph, what time will he take to cover 80 metres?']

In [39]:
concepts_list = concepts_df.name.tolist()

In [84]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    # output_parser=StrOutputParser(),
)    

# Define the prompt template for math concept tagging
template = """
You are an expert in math education. Your task is to identify the Common Core high school math conceptual categories that each given math word problem relates to. The possible categories are: {concepts_list}.
You will be given a math question, and you need to list all the relevant categories that apply to that question. You should not provide any explanations or reasoning, just the categories.

Here are some examples:

Example 1:
Question: Solve for x: 2x + 3 = 7
Categories: Expressions and Equations

Example 2:
Question: What is the area of a rectangle with length 5 and width 3?
Categories: Area and Volume

Example 3:
Question: If a train travels at 60 km/h, how far will it travel in 2 hours?
Categories: Dimensional Analysis, Operations with fractions and decimals

Now, for the following question, list all the categories that apply:

Question: {question}

Categories:
"""

# Create the prompt
prompt = PromptTemplate(template=template)

answers = []

test_df = questions_df.tail(10)

for i, rows in test_df.iterrows():
    question = rows.question
    question_id = rows.id

    message = prompt.format(
        concepts_list=concepts_list,
        question=question
    )

    response = llm.invoke(message)
    
    # add to the answers list
    answers.append({
        "question_id": question_id,
        "response": response,
    })
    time.sleep(2)
    
answers

[{'question_id': 88,
  'response': AIMessage(content='Expressions and Equations', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--52b724f7-5e58-47b5-b1bc-e84bef882c3b-0', usage_metadata={'input_tokens': 391, 'output_tokens': 4, 'total_tokens': 395, 'input_token_details': {'cache_read': 0}})},
 {'question_id': 89,
  'response': AIMessage(content='Real Number System', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--ef99f39e-0370-4d4d-8e56-fb89ef22ff15-0', usage_metadata={'input_tokens': 333, 'output_tokens': 4, 'total_tokens': 337, 'input_token_details': {'cache_read': 0}})},
 {'question_id': 90,
  'response': AIMessage(content='Real Number System', additional_kwargs={}, response_metadata={'prompt_fee

In [85]:
rough_top5_answers = pd.DataFrame(answers)
rough_top5_answers.to_pickle("rough_tail10_answers.pkl")

In [86]:
rough_top5_answers.loc[0, 'response'].content

'Expressions and Equations'

In [87]:
rough_top5_answers['response_content'] = rough_top5_answers['response'].apply(lambda x: x.content)
rough_top5_answers

,question_id,response,response_content
0,88,content='Expressions and Equations' additional...,Expressions and Equations
1,89,content='Real Number System' additional_kwargs...,Real Number System
2,90,content='Real Number System' additional_kwargs...,Real Number System
3,91,content='Expressions and Equations' additional...,Expressions and Equations
4,92,content='Probability' additional_kwargs={} res...,Probability
5,93,content='Expressions and Equations' additional...,Expressions and Equations
6,94,content='Expressions and Equations' additional...,Expressions and Equations
7,95,content='Expressions and Equations' additional...,Expressions and Equations
8,96,content='Expressions and Equations' additional...,Expressions and Equations
9,97,content='Area and Volume' additional_kwargs={}...,Area and Volume


In [88]:
# Split the 'response' column by the newline character and create a list of strings
rough_top5_answers['response_list'] = rough_top5_answers['response_content'].str.split('\n')
rough_top5_answers = rough_top5_answers.explode('response_list')
rough_top5_answers

,question_id,response,response_content,response_list
0,88,content='Expressions and Equations' additional...,Expressions and Equations,Expressions and Equations
1,89,content='Real Number System' additional_kwargs...,Real Number System,Real Number System
2,90,content='Real Number System' additional_kwargs...,Real Number System,Real Number System
3,91,content='Expressions and Equations' additional...,Expressions and Equations,Expressions and Equations
4,92,content='Probability' additional_kwargs={} res...,Probability,Probability
5,93,content='Expressions and Equations' additional...,Expressions and Equations,Expressions and Equations
6,94,content='Expressions and Equations' additional...,Expressions and Equations,Expressions and Equations
7,95,content='Expressions and Equations' additional...,Expressions and Equations,Expressions and Equations
8,96,content='Expressions and Equations' additional...,Expressions and Equations,Expressions and Equations
9,97,content='Area and Volume' additional_kwargs={}...,Area and Volume,Area and Volume


In [89]:
# join the concept_ids
df = rough_top5_answers.merge(
    concepts_df,
    left_on='response_list',
    right_on='name',
    how='left'
)

df.rename(columns={
    'question': 'question_id',
}, inplace=True)

In [90]:
df[['question_id','concept_id']].to_csv("rough_tail10_answers.csv", index=False)